### Algorithm: NoiseStra_PrivSize
Under stratified random sampling, give a zCDP CI for population mean by adding noise to both stratum sums and sample sizes. Thus, the sample sizes are considered private. 


In [ ]:
''' Same as the one in Algorithm: NoiseStra_PubSize'''
def make_map_partition_meas(meas):
    return Measurement(
        ProductDomain(m.input_domain for m in meas),
        ProductDomain(m.output_domain for m in meas),
        function(|data: Vec<DI::Carrier>|{
            output = []
            for part, m in zip(data, meas):
                output.append(m(part))
            return output
            
        }),
        ProductMetric(meas[0].input_metric),
        meas.output_measure,
        Privacy_Map(|d_in: MI::Distance|{
            return max(m.map(d_in) for m in meas)
        })
    )                 

In [ ]:
'''
    :param strat_sizes: the population size of each stratum
    :param scale_samp_sizes: scale of gaussian noise added to sample sizes
    :param scale_samp_sums: scale of gaussian noise added to sample sums
    :return private estimators for mean and variance. '''
def postprocess_noisestra_privsize(strat_sizes, scale_samp_sizes, scale_samp_sums):
    weights = strat_sizes / sum(strat_sizes)
    return make_postprocess(
        VectorDomain<ProductDomain<AllDomain<f64>>>
        VectorDomain<AllDomain<f64>>
        
        function(|noisy_sample_sizes_sums: Vec<Vec<f64>>) |{
            noisy_sample_sizes, noisy_sample_sums = noisy_sample_sizes_sums
            noisy_sample_sizes[noisy_sample_sizes < 2] = 2 
            noisy_strat_mean = min(max(noisy_sample_sums/noisy_sample_sizes, 0), 1)
            noisy_mean = sum(weights * noisy_stra_mean)
            noisy_strta_var = (strat_sizes - noisy_sample_sizes) / (strat_sizes - 1) * noisy_strat_mean * (1 - noisy_strat_mean) / noisy_sample_sizes + \ 
                              scale_samp_sums / noisy_sample_sizes ** 2 + noisy_strat_mean ** 2 * scale_samp_sizes / noisy_sample_sizes ** 2
            noisy_var = sum(weights * noisy_strat_var)
            return [noisy_mean, noisy_var]
        })
    


In [ ]:
def make_noisestra_privsize(strat_sizes, scale_samp_sizes, scale_samp_sums):
    meas_sums = make_map_partition_meas([
        make_bounded_sum((0,1)) >> lipschitz_cast((0, 2**MANTISSA_BITS), TI = int, TO = float) >> make_base_gaussian(scale_samp_sums)]
        * len(strat_sizes))
    
    meas_sizes = make_map_partition_meas([
        make_count((0,1)) >> lipschitz_cast((0, 2**MANTISSA_BITS), TI = int, TO = float) >> make_base_gaussian(scale_samp_sums)]
        * len(strat_sizes))
    
    return make_basic_composition([meas_sums, meas_sizes]) >> postprocess_noisestra_privsize(strat_sizes, scale_samp_sizes, scale_samp_sums)